In [1]:
!pip install fastapi uvicorn pyngrok streamlit transformers datasets requests python-multipart nest_asyncio
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 73.0 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 90.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 2.9 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled packaging-25.0
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. Thi

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
import numpy as np
import faiss
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModel, pipeline
from pyngrok import ngrok
import uvicorn
import nest_asyncio

nest_asyncio.apply()

app = FastAPI()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

@app.on_event("startup")
async def startup_event():
    global tokenizer, model, qa_pipe, index, contexts
    
    # Load model
    tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
    model = AutoModel.from_pretrained("distilbert-base-uncased").to(device)
    qa_pipe = pipeline('question-answering', 
                      model='huytranduck/distilbert-finetuned-squadv2',
                      device=0 if torch.cuda.is_available() else -1)
    
    # Load dataset và tạo index
    dataset = load_dataset('squad_v2', split='validation')
    dataset = dataset.filter(
        lambda x: len(x['answers']['text']) > 0
    )

    contexts = [ex['context'] for ex in dataset]
    
    # Tạo embeddings
    embeddings = []
    for batch in batch_generator(dataset['context'], 32):
        inputs = tokenizer(batch, return_tensors='pt', 
                         padding=True, truncation=True).to(device)
        with torch.no_grad():
            outputs = model(**inputs).last_hidden_state[:,0].cpu().numpy()
        embeddings.extend(outputs)
    
    index = faiss.IndexFlatL2(embeddings[0].shape[0])
    index.add(np.array(embeddings, dtype=np.float32))

def batch_generator(data, batch_size):
    for i in range(0, len(data), batch_size):
        yield data[i:i+batch_size]

class Query(BaseModel):
    question: str
    top_k: int = 3

@app.post("/search")
async def semantic_search(query: Query):
    # Tạo embedding cho câu hỏi
    inputs = tokenizer(query.question, return_tensors='pt').to(device)
    with torch.no_grad():
        q_embedding = model(**inputs).last_hidden_state[:,0].cpu().numpy()
    
    scores, indices = index.search(q_embedding.astype(np.float32), query.top_k)
    
    results = []
    for idx, score in zip(indices[0], scores[0]):
        context = contexts[idx]
        answer = qa_pipe(question=query.question, context=context)
        results.append({
            "score": float(score),
            "context": context,
            "answer": answer['answer'],
            "confidence": answer['score']
        })
    
    return {"results": results}

ngrok.set_auth_token("NGROK_AUTH_TOKEN")
public_url = ngrok.connect(8000).public_url
print(f"Public API URL: {public_url}")

if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)

2025-05-14 17:17:29.261379: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747243049.459767      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747243049.514823      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


/tmp/ipykernel_35/1150266466.py:18: DeprecationWarning: 
        on_event is deprecated, use lifespan event handlers instead.

        Read more about it in the
        [FastAPI docs for Lifespan Events](https://fastapi.tiangolo.com/advanced/events/).
        
  @app.on_event("startup")


Public API URL: https://096e-34-73-105-169.ngrok-free.app                                           


INFO:     Started server process [35]
INFO:     Waiting for application startup.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

...

Filter:   0%|          | 0/11873 [00:00<?, ? examples/s]

INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     113.23.123.13:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     2405:4802:1ca2:f0d0:4160:ac35:e0fe:7fd3:0 - "POST /search HTTP/1.1" 200 OK
INFO:     2405:4802:1ca2:f0d0:4160:ac35:e0fe:7fd3:0 - "POST /search HTTP/1.1" 200 OK
INFO:     2405:4802:1ca2:f0d0:4160:ac35:e0fe:7fd3:0 - "POST /search HTTP/1.1" 200 OK


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


INFO:     2405:4802:1ca2:f0d0:4160:ac35:e0fe:7fd3:0 - "POST /search HTTP/1.1" 200 OK
INFO:     2405:4802:1ca2:f0d0:4160:ac35:e0fe:7fd3:0 - "POST /search HTTP/1.1" 200 OK
INFO:     2405:4802:1ca2:f0d0:4160:ac35:e0fe:7fd3:0 - "POST /search HTTP/1.1" 200 OK
INFO:     2405:4802:1ca2:f0d0:4160:ac35:e0fe:7fd3:0 - "POST /search HTTP/1.1" 200 OK
INFO:     2405:4802:1ca2:f0d0:4160:ac35:e0fe:7fd3:0 - "POST /search HTTP/1.1" 200 OK
INFO:     2405:4802:1ca2:f0d0:4160:ac35:e0fe:7fd3:0 - "POST /search HTTP/1.1" 200 OK
INFO:     2405:4802:1ca2:f0d0:4160:ac35:e0fe:7fd3:0 - "POST /search HTTP/1.1" 200 OK
INFO:     2405:4802:1ca2:f0d0:4160:ac35:e0fe:7fd3:0 - "POST /search HTTP/1.1" 200 OK
INFO:     2405:4802:1ca2:f0d0:4160:ac35:e0fe:7fd3:0 - "POST /search HTTP/1.1" 200 OK
INFO:     2405:4802:1ca2:f0d0:4160:ac35:e0fe:7fd3:0 - "POST /search HTTP/1.1" 200 OK
INFO:     2405:4802:1ca2:f0d0:4160:ac35:e0fe:7fd3:0 - "POST /search HTTP/1.1" 200 OK
